# DATA PREPROCESSING

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [53]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen= ImageDataGenerator(rescale=1./255)

In [54]:
x_train = train_datagen.flow_from_directory(r'C:\Users\saira\Downloads\ECG_Arrhythmia_Dataset\ECG_Datasets\Training_dataset',target_size = (64, 64),batch_size = 32,class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r'C:\Users\saira\Downloads\ECG_Arrhythmia_Dataset\ECG_Datasets\Test_dataset',target_size = (64, 64),batch_size = 32,class_mode = 'categorical')

Found 2600 images belonging to 5 classes.
Found 650 images belonging to 5 classes.


In [55]:
x_train.class_indices

{'Fusion_beat': 0,
 'Normal_beat': 1,
 'Supraventricular_ectopic_beat': 2,
 'Unknown_beat': 3,
 'Ventricular_ectopic_beat': 4}

# MODEL BUILDING

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [6]:
model=Sequential()

In [7]:
model.add(Convolution2D(48,(3,3),input_shape=(64,64,3),activation="relu"))

In [8]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [9]:
model.add(Flatten())

In [10]:
model.add(Dense(kernel_initializer="random_uniform",activation="relu",units=150))

In [11]:
model.add(Dense(kernel_initializer="random_uniform",activation="softmax",units=5))

In [12]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 48)        1344      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 48)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 46128)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               6919350   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 755       
Total params: 6,921,449
Trainable params: 6,921,449
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit_generator(x_train, steps_per_epoch=500, epochs=12,validation_data=x_test)


Epoch 1/12
500/500 [==============================] - 92s 184ms/step - loss: 1.0261 - accuracy: 0.5920 - val_loss: 2.2800 - val_accuracy: 0.3400
Epoch 2/12
500/500 [==============================] - 99s 197ms/step - loss: 0.4528 - accuracy: 0.8437 - val_loss: 3.1406 - val_accuracy: 0.4169
Epoch 3/12
500/500 [==============================] - 94s 188ms/step - loss: 0.3444 - accuracy: 0.8856 - val_loss: 4.0933 - val_accuracy: 0.4154
Epoch 4/12
500/500 [==============================] - 93s 187ms/step - loss: 0.2832 - accuracy: 0.9033 - val_loss: 3.1259 - val_accuracy: 0.4246
Epoch 5/12
500/500 [==============================] - 98s 196ms/step - loss: 0.2345 - accuracy: 0.9219 - val_loss: 4.4734 - val_accuracy: 0.4123
Epoch 6/12
500/500 [==============================] - 99s 199ms/step - loss: 0.1906 - accuracy: 0.9358 - val_loss: 4.2991 - val_accuracy: 0.4015
Epoch 7/12
500/500 [==============================] - 101s 202ms/step - loss: 0.1558 - accuracy: 0.9472 - val_loss: 3.7096 - val_

In [33]:
model.save('ECG1.h5')

# PREDICTION

In [34]:
from keras.models import load_model
import numpy as np

In [35]:
model = load_model('ECG1.h5')
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

from skimage.transform import resize

def detect(frame):
    try:
        beat_type=['Fusion beat','Normal beat','Supraventricular ectopic beat','unknown beat','Ventricular ectopic beat']
        img = resize(frame,(64,64))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)
        print(prediction)
        prediction = model.predict_classes(img)
        print(beat_type[prediction[0]])
    except AttributeError:
        print("shape not found")

In [36]:
import cv2
path=r'C:\Users\saira\Downloads\ECG_Arrhythmia_Dataset\ECG_Datasets\Training_dataset\Normal_beat\fig_1.png'

In [37]:
frame=cv2.imread(r'C:\Users\saira\Downloads\ECG_Arrhythmia_Dataset\ECG_Datasets\Training_dataset\Normal_beat\fig_1.png')
data=detect(frame)

[[4.2687338e-03 9.5976985e-01 3.5652630e-02 1.8997461e-04 1.1892650e-04]]
Normal beat


In [38]:
path=r'C:\Users\saira\Downloads\ECG_Arrhythmia_Dataset\ECG_Datasets\Training_dataset\Fusion_beat\Ffig_1.png'

In [39]:
frame=cv2.imread(path)
data=detect(frame)

[[9.6088248e-01 5.4822024e-04 3.3325743e-02 7.8596349e-06 5.2356790e-03]]
Fusion beat


In [40]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Unknown_beat\ffig_31200001.png'
data=detect(cv2.imread(path))

[[2.3596624e-03 3.6680620e-04 7.8853351e-05 9.9694246e-01 2.5213513e-04]]
unknown beat


In [41]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Unknown_beat\ffig_31200011.png'
data=detect(cv2.imread(path))

[[1.6034318e-02 6.0373060e-07 3.5131990e-04 9.8360044e-01 1.3263278e-05]]
unknown beat


In [42]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Ventricular_ectopic_beat\Efig_93.png'
data=detect(cv2.imread(path))

[[1.9661039e-02 2.0973082e-05 7.7328958e-05 5.0255470e-02 9.2998523e-01]]
Ventricular ectopic beat


In [43]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Ventricular_ectopic_beat\Efig_105.png'
data=detect(cv2.imread(path))

[[2.7690907e-03 1.1138022e-05 1.9808274e-05 5.1752822e-03 9.9202466e-01]]
Ventricular ectopic beat


In [44]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Supraventricular_ectopic_beat\Afig_5.png'
data=detect(cv2.imread(path))

[[0.06858845 0.26511472 0.63532966 0.01184977 0.01911746]]
Supraventricular ectopic beat


In [45]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Training_dataset\Supraventricular_ectopic_beat\Afig_1.png'
data=detect(cv2.imread(path))

[[2.47881771e-03 8.74465704e-01 1.22889765e-01 1.91033359e-05
  1.46539867e-04]]
Normal beat


In [46]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Test_dataset\Fusion_beat\Ffig_652.png'
data=detect(cv2.imread(path))

[[9.9559832e-01 5.1753165e-07 5.7018191e-05 1.8754833e-04 4.1566365e-03]]
Fusion beat


In [48]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Test_dataset\Supraventricular_ectopic_beat\Afig_2234.png'
data=detect(cv2.imread(path))

[[0.08758871 0.04773206 0.79840034 0.00197967 0.06429926]]
Supraventricular ectopic beat


In [50]:
path=r'C:\Users\saira\OneDrive\Desktop\intership\ECG_Datasets\Test_dataset\Ventricular_ectopic_beat\Efig_60.png'
data=detect(cv2.imread(path))

[[6.1305072e-03 1.9345645e-04 3.0032868e-04 4.7417480e-01 5.1920092e-01]]
Ventricular ectopic beat
